In [ ]:
#clear all
%reset -f

#import packages
import numpy as np
import scipy
import sys
import os
import pandas as pd
import mne
import matplotlib
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'frn'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [ ]:
# First, we get the 800 timepoints we are considering as a list, to use for indices given by cluster-based permutation later
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')
pp = 0 #only need one participant

# we can use aligned data

id_directory = os.path.join(data_directory, 'p%03d/' % pp)
pp_directory = os.path.join(id_directory, erps)
fname = os.path.join(pp_directory, 'p%03d_%s_%s-ave.fif' % (pp, 'early_late', 'aligned'))
evoked = mne.read_evokeds(fname)
df = evoked[0].to_data_frame()
time = df['time'].tolist()

In [ ]:
# Load data for each condition
# Take the mean across electrodes of interest per participant
# transform data into array accepted for cluster-based permutation test in mne
# output: (n_participants, n_timepts) for each condition

root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#specify channels we need - FRN
channels = ['FCz', 'F3', 'Fz', 'F4', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4']

    
#read in evoked object
tasknames = ['rot', 'rdm', 'mir']
conditionnames = ['early', 'late']


#aligned
evoked_list = []

for pp in participants:
    id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
    pp_directory = os.path.join(id_directory, erps)
    fname = os.path.join(pp_directory, 'p%03d_%s_%s-ave.fif' % (participants[pp], 'early_late', 'aligned'))
    evoked = mne.read_evokeds(fname)
    evoked = evoked[0]
    evoked = evoked.get_data(picks=channels) #will give data of shape (n_channels, n_timepts)
    evoked = evoked.mean(axis=0) #take mean of cols or the channels we picked
    evoked_list.append(evoked)
    
aligned_flist = evoked_list

#rot and rdm and mir
for task in range(0, len(tasknames)):
    for condition in range(0, len(conditionnames)):
        evoked_list = []
        for pp in participants:
            id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
            pp_directory = os.path.join(id_directory, erps)
            fname = os.path.join(pp_directory, 'p%03d_%s_%s-ave.fif' % (participants[pp], conditionnames[condition], tasknames[task]))
            evoked = mne.read_evokeds(fname)
            evoked = evoked[0]
            evoked = evoked.get_data(picks=channels) #will give data of shape (n_channels, n_timepts)
            evoked = evoked.mean(axis=0) #take mean of cols or the channels we picked
            evoked_list.append(evoked)
        if task == 0 and condition == 0:
            early_rot_flist = evoked_list
        elif task == 0 and condition == 1:
            late_rot_flist = evoked_list
        elif task == 1 and condition == 0:
            early_rdm_flist = evoked_list
        elif task == 1 and condition == 1:
            late_rdm_flist = evoked_list
        elif task == 2 and condition == 0:
            early_mir_flist = evoked_list
        elif task == 2 and condition == 1:
            late_mir_flist = evoked_list

In [ ]:
# Comparing two ERP signals is just the same as taking their difference (ERP1 minus ERP2) and using this in permutation test
def get_clust_perm_test(conditionA, conditionB, pval, n_permutations):
    #take difference of two conditions
    data = np.subtract(conditionA, conditionB)
    np.shape(data)
    #define cluster forming threshold based on p-value
    df = len(participants) - 1
    thresh = scipy.stats.t.ppf(1 - pval / 2, df)
    #run cluster-based permutation test
    T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh, 
                                                          n_permutations = n_permutations, tail = 0, 
                                                          adjacency = None, seed = 999, 
                                                          out_type = 'mask', verbose = True)

    return T_0, clust_idx, clust_pvals, H0

In [ ]:
# First, we can compare each condition to aligned
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values
# This information can then be included in ERP plots

flists = [early_rot_flist, late_rot_flist, early_rdm_flist, late_rdm_flist, early_mir_flist, late_mir_flist]
conditionnames = ['earlyrot', 'laterot', 'earlyrdm', 'laterdm', 'earlymir', 'latemir']
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

for f in range(0, len(flists)):
    T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(flists[f], aligned_flist, p, perms)
#     print(clust_idx, clust_pvals)
    for clust in range(0, len(clust_idx)):
        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
        cluster_start = cluster.start
        clust_idx_start.append(cluster_start)
        
        cluster_end = cluster.stop
        clust_idx_end.append(cluster_end)
        
        time_idx_start = time[cluster_start]
        time_start.append(time_idx_start)
        
        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
        time_end.append(time_idx_end)
        
        clust_p = clust_pvals[clust]
        p_values.append(clust_p)
        
        condition.append(conditionnames[f])
        
perm_test = pd.DataFrame(
    {'condition': condition,
     'clust_idx_start': clust_idx_start,
     'clust_idx_end': clust_idx_end,
     'time_start': time_start,
     'time_end': time_end,
     'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Permutation_test_vsAligned_%s.csv' % (erps))
perm_test.to_csv(perm_test_filename)

In [ ]:
# Next, we subtract aligned from each condition, so that we can compare early vs late in each perturbation type
        

In [ ]:
#plot FRN/ERN here using MNE functions
#channels = ['FCz', 'Fz']
channels = ['FCz', 'F3', 'Fz', 'F4', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4']
#channels = ['C3']
#channels = ['C4']
# channels = ['Oz','O1','O2']
evokeds = dict(aligned = aligned_flist, 
               early_rot = early_rot_flist,
               late_rot = late_rot_flist, 
               early_rdm = early_rdm_flist, 
               late_rdm = late_rdm_flist,
               early_mir = early_mir_flist,
               late_mir = late_mir_flist)
#plot
mne.viz.plot_compare_evokeds(evokeds, picks=channels, combine = 'mean', legend = 'lower right', ylim=dict(eeg=[-15, 5]), ci=True)

In [ ]:
#calculate difference waves for FRN/ERN
diffconds = ['earlyrot', 'laterot', 'earlyrdm', 'laterdm', 'earlymir', 'latemir']
earlyrot_diff = []
laterot_diff = []
earlyrdm_diff = []
laterdm_diff = []
earlymir_diff = []
latemir_diff = []

for cond in range(0, len(diffconds)):
    for pp in participants:
        if cond == 0:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['early_rot'][pp]], 
                                                            weights=[-1, 1])
            earlyrot_diff.append(diffevks)
        elif cond == 1:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['late_rot'][pp]], 
                                                         weights=[-1, 1])
            laterot_diff.append(diffevks)
        elif cond == 2:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['early_rdm'][pp]], 
                                                         weights=[-1, 1])
            earlyrdm_diff.append(diffevks)
        elif cond == 3:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['late_rdm'][pp]], 
                                                         weights=[-1, 1])
            laterdm_diff.append(diffevks)
        elif cond == 4:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['early_mir'][pp]], 
                                                         weights=[-1, 1])
            earlymir_diff.append(diffevks)
        elif cond == 5:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['late_mir'][pp]], 
                                                         weights=[-1, 1])
            latemir_diff.append(diffevks)


In [ ]:
#saved difference waves as data frame for FRN/ERN

difflists = ['earlyrot', 'laterot', 'earlyrdm', 'laterdm', 'earlymir', 'latemir']

for diff in difflists:
    newdf = pd.DataFrame()
    for pp in participants:
        if diff == 'earlyrot':
            df = earlyrot_diff[pp].to_data_frame()
        elif diff == 'laterot':
            df = laterot_diff[pp].to_data_frame()
        elif diff == 'earlyrdm':
            df = earlyrdm_diff[pp].to_data_frame()
        elif diff == 'laterdm':
            df = laterdm_diff[pp].to_data_frame()
        elif diff == 'earlymir':
            df = earlymir_diff[pp].to_data_frame()
        elif diff == 'latemir':
            df = latemir_diff[pp].to_data_frame()
             
        time = df['time'].tolist()
        df = df[['FCz', 'F3', 'Fz', 'F4', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4']]
        ppdat = []
        for timept in range(0,800):
            subdf = df.loc[timept,:].mean()
            ppdat.append(subdf)
    
        newdf['pp%03d'% pp] = ppdat
    
    newdf['time'] = time
    newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Diffwaves_DF_%s_%s.csv' % (diff, erps))
    newdf.to_csv(newdf_filename)
        


In [ ]:
#plot difference waves for FRN/ERN here in MNE
mne.viz.plot_compare_evokeds({'Earlyrot - Aligned':earlyrot_diff, 'Laterot - Aligned':laterot_diff, 'Earlyrdm - Aligned':earlyrdm_diff, 'Laterdm - Aligned':laterdm_diff, 'Earlymir - Aligned':earlymir_diff, 'Latemir - Aligned':latemir_diff,}, 
                             picks=channels, show_sensors='upper right',
                             combine='mean',
                             title='Difference Wave');

In [ ]:
#runs for LRP ONLY: need separate files for C3 and C4

flists = ['aln', 'earlyrot', 'laterot', 'earlyrdm', 'laterdm', 'earlymir', 'latemir']
channels = ['C3', 'C4']
for channel in channels:
    for f in flists:
        newdf = pd.DataFrame()
        for pp in participants:
            if f == 'aln':
                df = aligned_flist[pp].to_data_frame()
            elif f == 'earlyrot':
                df = early_rot_flist[pp].to_data_frame()
            elif f == 'laterot':
                df = late_rot_flist[pp].to_data_frame()
            elif f == 'earlyrdm':
                df = early_rdm_flist[pp].to_data_frame()
            elif f == 'laterdm':
                df = late_rdm_flist[pp].to_data_frame()
            elif f == 'earlymir':
                df = early_mir_flist[pp].to_data_frame()
            elif f == 'latemir':
                df = late_mir_flist[pp].to_data_frame()
             
            time = df['time'].tolist()
            df = df[[channel]]
            ppdat = []
            for timept in range(0,800):
                subdf = df.loc[timept,:].mean()
                ppdat.append(subdf)
    
            newdf['pp%03d'% pp] = ppdat
    
        newdf['time'] = time
        newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s_%s.csv' % (f, erps, channel))
        newdf.to_csv(newdf_filename)

In [ ]:
# Difference waves between aligned and perturbed conditions for C3
channels = ['C3']
#channels = ['C4']
# channels = ['Oz','O1','O2']
evokeds = dict(aligned = aligned_flist, 
               early_rot = early_rot_flist,
               late_rot = late_rot_flist, 
               early_rdm = early_rdm_flist, 
               late_rdm = late_rdm_flist,
               early_mir = early_mir_flist,
               late_mir = late_mir_flist)
#plot
mne.viz.plot_compare_evokeds(evokeds, picks=channels, combine = 'mean', legend = 'lower right', ylim=dict(eeg=[-15, 5]), ci=True)

In [ ]:
#calculate difference waves for LRP C3
diffconds = ['earlyrot', 'laterot', 'earlyrdm', 'laterdm', 'earlymir', 'latemir']
earlyrot_diff = []
laterot_diff = []
earlyrdm_diff = []
laterdm_diff = []
earlymir_diff = []
latemir_diff = []

for cond in range(0, len(diffconds)):
    for pp in participants:
        if cond == 0:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['early_rot'][pp]], 
                                                            weights=[-1, 1])
            earlyrot_diff.append(diffevks)
        elif cond == 1:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['late_rot'][pp]], 
                                                         weights=[-1, 1])
            laterot_diff.append(diffevks)
        elif cond == 2:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['early_rdm'][pp]], 
                                                         weights=[-1, 1])
            earlyrdm_diff.append(diffevks)
        elif cond == 3:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['late_rdm'][pp]], 
                                                         weights=[-1, 1])
            laterdm_diff.append(diffevks)
        elif cond == 4:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['early_mir'][pp]], 
                                                         weights=[-1, 1])
            earlymir_diff.append(diffevks)
        elif cond == 5:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['late_mir'][pp]], 
                                                         weights=[-1, 1])
            latemir_diff.append(diffevks)

In [ ]:
#saved difference waves as data frame for LRP C3

difflists = ['earlyrot', 'laterot', 'earlyrdm', 'laterdm', 'earlymir', 'latemir']

for diff in difflists:
    newdf = pd.DataFrame()
    for pp in participants:
        if diff == 'earlyrot':
            df = earlyrot_diff[pp].to_data_frame()
        elif diff == 'laterot':
            df = laterot_diff[pp].to_data_frame()
        elif diff == 'earlyrdm':
            df = earlyrdm_diff[pp].to_data_frame()
        elif diff == 'laterdm':
            df = laterdm_diff[pp].to_data_frame()
        elif diff == 'earlymir':
            df = earlymir_diff[pp].to_data_frame()
        elif diff == 'latemir':
            df = latemir_diff[pp].to_data_frame()
             
        time = df['time'].tolist()
        #df = df[['FCz', 'F3', 'Fz', 'F4', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4']]
        df = df[['C3']]
        ppdat = []
        for timept in range(0,800):
            subdf = df.loc[timept,:].mean()
            ppdat.append(subdf)
    
        newdf['pp%03d'% pp] = ppdat
    
    newdf['time'] = time
    newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Diffwaves_DF_%s_%s_C3.csv' % (diff, erps))
    newdf.to_csv(newdf_filename)
        

In [ ]:
#plot difference waves for LRP C3 here in MNE
mne.viz.plot_compare_evokeds({'Earlyrot - Aligned':earlyrot_diff, 'Laterot - Aligned':laterot_diff, 'Earlyrdm - Aligned':earlyrdm_diff, 'Laterdm - Aligned':laterdm_diff, 'Earlymir - Aligned':earlymir_diff, 'Latemir - Aligned':latemir_diff}, 
                             picks=channels, show_sensors='upper right',
                             combine='mean',
                             title='Difference Wave');